In [36]:
import pandas as pd
import re
import numpy as np

# Read CSV
panel_df = pd.read_csv("data/panel_data.csv")



In [39]:
import numpy as np
import pandas as pd

# Make sure these exist
required = ["pubid","year","age","teen","teen_move","income_wages"]
missing = [c for c in required if c not in panel_df.columns]
print("Missing:", missing)

# Clean: set any negative codes to NaN for numeric columns (if not already done)
num_cols = panel_df.select_dtypes(include=["int64","float64","int32","float32"]).columns
panel_df[num_cols] = panel_df[num_cols].mask(panel_df[num_cols] < 0)

# Adult outcome sample (you can tweak)
dfm = panel_df.copy()
dfm = dfm[(dfm["age"] >= 25) & (dfm["age"] <= 40)]  # adult window
dfm = dfm[dfm["income_wages"].notna()]               # need outcome observed

# Log wages (standard)
dfm["log_wage"] = np.log(dfm["income_wages"] + 1)


Missing: []


In [ ]:
import statsmodels.formula.api as smf

controls = [
    "sex",
    "race_black", "race_hispanic", "race_mixed",
    "cv_hgc", "cv_highest_degree",   # if you kept these names; adjust if needed
    "cv_msa", "cv_urban-rural",      # adjust if your cols are in_msa / urban_rural
]

# keep only controls that exist
controls = [c for c in controls if c in dfm.columns]
control_str = " + ".join(controls) if controls else "1"

formula = f"log_wage ~ teen_move + {control_str} + C(year)"
ols = smf.ols(formula, data=dfm).fit(cov_type="HC1")
print(ols.summary())
